# Generación de Custom Mosaics para el experimento 9
* **Autor:** Julián Zuloaga
* **Asignatura:** Memoria de Título
* **Fecha:** 11 de octubre de 2022
---

**Descripción:** Este programa sirve para generar Custom Mosaics similares a una multitud con las imágenes de la base de datos AffectNet en formato YOLOv4. Además:
* Se agregaron ejemplos negativos, 10 por mosaico, con Horizontal Flip.
* Se puso 7 rostros en 3 distintas resoluciones por mosaico.

## Paso 1: Carga de recursos necesarios

Para el funcionamiento de este programa es necesario importar algunas librerías, crear algunos directorios, descargar algunos archivos, entre otros.

In [ ]:
# Se importan librerías
from IPython.display import clear_output # limpia salida
import os # manejo de directorios
import random # para funciones de randomizado
from PIL import Image, ImageDraw # para manejo de imágenes
import numpy as np # manejo de arrays de numpy
from google.colab.patches import cv2_imshow # para mostrar imágen generada
import cv2 # para visualización y agregar texto
import PIL

In [ ]:
# Se crean carpetas donde se descargará la base de datos (obj.zip y test.zip)
!mkdir -v ./AffectNet_dataset
!mkdir -v ./Ejemplos_negativos

In [ ]:
# Se monta unidad de google drive a la máquina virtual de Colab
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Se crea un symbolic link para reemplazar "/content/gdrive/My\ Drive/" a "/mydrive"
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
# Se va a la carpeta /content/
%cd /content/

In [ ]:
# Se copia la base de datos AffectNet en formato YOLOv4 desde GDrive a Colab.
!cp /mydrive/Proc_AffectNet_YOLOv4_v1/obj.zip ./AffectNet_dataset/obj.zip
!cp /mydrive/Proc_AffectNet_YOLOv4_v1/test.zip ./AffectNet_dataset/test.zip
# Se descargan los ejemplos negativos:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1377C02YttAOMGNwyZjkK78M_2wPTGU8q' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1377C02YttAOMGNwyZjkK78M_2wPTGU8q" -O ./Ejemplos_negativos/obj.zip && rm -rf /tmp/cookies.txt
# Se verifica descarga correcta
check1 = os.path.isfile('./AffectNet_dataset/obj.zip')
check2 = os.path.isfile('./AffectNet_dataset/test.zip')
if check1 == True and check2 == True:
  print("[INFO]: Base de datos descargada correctamente.")
else:
  print("[ERROR]: La base de datos no se ha descargado correctamente!")
  assert False

In [ ]:
# Se descomprime los archivos zip de la base de datos
!mkdir /content/AffectNet_dataset/obj/
!mkdir /content/AffectNet_dataset/test/
!unzip -q ./AffectNet_dataset/obj.zip -d ./AffectNet_dataset/obj/
!unzip -q ./AffectNet_dataset/test.zip -d ./AffectNet_dataset/test/
# Se descomprime archivo .zip con ejemplos negativos
!unzip -q ./Ejemplos_negativos/obj.zip -d ./Ejemplos_negativos/
# Se borran los archivos comprimidos
!rm ./AffectNet_dataset/obj.zip
!rm ./AffectNet_dataset/test.zip
# Una vez descomprimidos los archivos, se borra el archivo .zip
!rm ./Ejemplos_negativos/obj.zip

In [ ]:
# Se aplica volveo horizontal a los ejemplos negativos
from PIL import Image, ImageOps
import PIL
import glob
img_list_ejemp = [x for x in glob.glob("/content/Ejemplos_negativos/*.jpg")]
img_list_ejemp.sort() # se ordena la lista, aunque no es necesario
contador = 0 # contador para evitar nombres repetidos
for neg_img in img_list_ejemp:
  this_img = Image.open(neg_img) # se abre imagen de ejemplo negativo
  this_img_mirror=ImageOps.mirror(this_img) # se aplica horizontal flip
  this_img_mirror.save(f"/content/Ejemplos_negativos/flip_{contador}.jpg") # se guarda imagen volteada
  this_img.close() # se cierra imagen
  this_img_mirror.close() # se cierra imagen con horizontal flip
  contador += 1 # se incrementa contador

## Paso 2: Generación de imágenes con Data Augmentation
En esta sección se generan las imágenes de mosaicos.

### 2.1. Se desordenan las imágenes
Se crea una lista con todas las imágenes de conjunto de entrenamiento y se reordena de manera pseudo-aleatoria para que las imágenes no se ordenen por clase durante la generación de los mosaicos.

In [ ]:
# Se genera lista con nombres de archivos en conjunto de entrenamiento
obj_img_list = list() # se inicializa lista para imágenes
# Se agregan los nombres de las imágenes a la lista
for file in os.listdir("./AffectNet_dataset/obj/"):
    if file.endswith(".jpg"):
        obj_img_list.append(file)
# Se imprime número de imágenes encontradas en conjunto de entrenamiento
print(f'[INFO]: Número de imágenes encontradas en set de entrenamiento: {len(obj_img_list)}')
# Se reordena la lista con nombres de manera aleatoria
random.seed(0) # se genera semilla para obtener siempre el mismo resultado
random.shuffle(obj_img_list) # se reordena la lista de manera aleatoria

In [ ]:
# Se genera lista con nombres de archivos en conjunto de ejemplos negativos
neg_img_list = list() # se inicializa lista de imágenes
# Se agergan los nombres de las imágenes a la lista
for file in os.listdir("./Ejemplos_negativos/"):
  if file.endswith(".jpg"):
    neg_img_list.append(file)

# Se imprime número de imágenes encontradas en conjunto de ejemplos negativos
print(f'[INFO]: Número de imágenes encontradas en set de ejemplos negativos: {len(neg_img_list)}')
# Se reordena la lista con nombres de manera aleatoria
random.seed(0) # se genera semilla para obtener siempre el mismo resultado
random.shuffle(neg_img_list) # se reordena la lista de manera aleatoria

### 2.2. Se selecciona imágenes y se calcula sus parámetros

In [ ]:
# Se define función auxiliar para detectar colisiones entre imágenes
from collections import namedtuple
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')
def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy

In [ ]:
# Se crea contador para llevar cuenta del número de imágenes restantes en el conjunto de entrenamiento
img_restantes = len(obj_img_list)
# Se establece número de imágenes por mosaico
n_img_por_mosaico = 7 # este valor debe ser ingresado por el usuario
# Se establece resolución de imagen de salida
res_mosaico = [1024,768] # [width, height]
# Aplicar máscara?
do_Mask = True

In [ ]:
# Se crea carpeta donde se guardará la nueva base de datos
!mkdir ./Mosaic_Dataset/
%cd ./Mosaic_Dataset/
!mkdir ./obj/
!mkdir ./test/

### 2.3. Procesamiento conjunto de entrenamiento

Se genera la nueva base de datos para entrenamiento en base al conjunto pre definido en la base de datos descargada (archivo obj.zip)

In [ ]:
# Se cambia directorio para generación de conjunto de entrenamiento.
%cd ./obj/

In [ ]:
# Se detiene ejecución automática de celdas
assert False

In [ ]:
from IPython.core.display import Javascript
# Se definen tamaños máximos y mínimos para las imágenes de rostros
max_face_res = [90, 90] # [ancho, alto]
min_face_res = [25, 25] # [ancho, alto]
# Proporción de imagenes por grupo:
size_L = 0.1                    # grupo grande
size_M = 0.32                   # grupo mediano
size_S = (1 - size_L - size_M)  # grupo pequeño
# Se calcula el número de imágenes para la proporción definida
n_L = 1
n_M = 1
n_S = 1
# Proporción del fondo por sección (S, M, L)
b_size_L = 0.3
b_size_M = 0.3
b_size_S = (1 - b_size_L - b_size_M)
# Se calcula el número de píxels verticales del fondo que corresponde a cada proporción
b_L = int(b_size_L*res_mosaico[1])
b_M = int(b_size_M*res_mosaico[1])
b_S = res_mosaico[1] - b_L - b_M

# Se inicializa lista con registro de coordenadas
img_reg = list()
neg_img_counter = 0

# Se inicializa variable contadora de imágenes generadas
j = 0
# Se comienza con la generación de imágenes 
while j < len(obj_img_list):

  # se crea archivo de texto para guardar anotaciones (etiqueta)
  background_label = open(f'{j}.txt', 'w')

  # Se crea fondo negro
  background = Image.new('RGB', res_mosaico, (0, 0, 0))

  #--------------------EJEMPLOS NEGATIVOS------------------------------------------------------------------
  # Se define número de ejemplos negativos a agregar por mosaico generado
  n_ejem_neg_por_mosaic = 10

  # Se inicializa lista con registro de vértices de imágenes ya colocadas
  img_reg = [(0, 0, 2, 2)]

  # Se agregan los ejemplos negativos de manera aleatória
  for ji in range(n_ejem_neg_por_mosaic):
    # si el contador igualó o sobrepasó el máximo, se reinicia
    if ji+neg_img_counter >= len(neg_img_list): neg_img_counter = 0
    # Se abre imagen de ejemplo negativo a agregar
    ejem_neg = Image.open(f'/content/Ejemplos_negativos/{neg_img_list[ji + neg_img_counter]}')
    
    do_intersect = True # variable de verificación
    # Se genera posición nueva y se verifica la no intersección
    while do_intersect:
        # Se genera propuesta de coordenadas aleatoriamente
        neg_pos_x = random.randint(0, res_mosaico[0])
        neg_pos_y = random.randint(0, b_S + b_M)
        # Se define vértice inferior derecho de la img
        v_inf_der = (neg_pos_x, neg_pos_y)
        # Se calcula el vértice superior izquierdo de la img
        v_sup_izq = (neg_pos_x - ejem_neg.size[0], neg_pos_y - ejem_neg.size[1])
        # Se define padding adicional a la imagen
        padding = 4 # píxeles
        # Se detecta intersección entre otras imágenes
        c2 = (v_sup_izq[0] - padding, v_sup_izq[1] - padding, v_inf_der[0] + padding , v_inf_der[1] + padding)
        d2 = Rectangle(c2[0],c2[1],c2[2],c2[3])
        for old_coord in img_reg:
          c1 = np.asarray(old_coord)
          d1 = Rectangle(c1[0],c1[1],c1[2],c1[3])
          #Se evalúa si la intersección entre ambas imágenes es distinto de vacío
          if area(d1,d2) is None:
            do_intersect = False # No se detectó intersección
          else:
            do_intersect = True # Se detectó intersección
            n_colisiones = n_colisiones + 1
            break # se detiene ciclo for y se vuelve a generar coordenadas
        # Una vez se verifició que no hay colisión, se guarda coordenadas en registro
        #img_reg.append(c2)

    # Se agrega el ejemplo negativo al fondo
    background.paste(ejem_neg, (neg_pos_x, neg_pos_y))
    # Se cierra imagen abierta con ejemplo negativo
    ejem_neg.close()
  
  # se agrega el número de ejemplos negatios agregados al contador
  neg_img_counter = neg_img_counter + n_ejem_neg_por_mosaic
  #-----------------------EJEMPLOS NEGATIVOS--------------------------------------------------------------

  # Contador de colisiones
  n_colisiones = 0
  # Se verifica si se llegó a las últimas imágenes
  if (len(obj_img_list) - j) < n_img_por_mosaico:
    n_img_por_mosaico = len(obj_img_list) - j # se reduce número de rostros por imagen generada
  
  # Se agregan las fotos con rostros al fondo
  for i in range(n_img_por_mosaico):
    # Se abre imagen con rostro
    face = Image.open(f'/content/AffectNet_dataset/obj/{obj_img_list[i+j]}')

    for ii in range(3):
      if ii < n_L:
        # número de imágenes por sección
        n_img = n_L
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = max_face_res[0]
        yface_min = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = max_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L)
        y_max = (res_mosaico[1] - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      elif ii >= n_L and ii < (n_M + n_L): 
        # número de imágenes por sección
        n_img = n_M
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L - b_M)
        y_max = (res_mosaico[1] - b_L - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      else:
        # número de imágenes por sección
        n_img = n_S
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = min_face_res[0]
        xface_max = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = min_face_res[1]
        yface_max = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = 0
        y_max = (res_mosaico[1] - b_L - b_M - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      # Se calcula nueva dimensión de imagen con rostro
      new_x_size = random.randint(xface_min, xface_max)
      #new_y_size = random.randint(yface_min, yface_max) # se pierde aspect ratio :(
      #Se calcula el alto para conservar aspect ratio
      wpercent = (new_x_size/float(face.size[0]))
      new_y_size = int((float(face.size[1])*float(wpercent)))
      # Se escala la imagen seleccionada
      face = face.resize((new_x_size, new_y_size), resample=PIL.Image.BILINEAR)
      # Se calcula posición de la imagen asegurando la no intersección con imágenes anteriores
      do_intersect = True # variable de verificación
      # Se genera posición nueva y se verifica la no intersección
      while do_intersect:
        # generación de coordenadas nuevas
        new_x_pos = random.randint(x_min, x_max)
        new_y_pos = random.randint(y_min, y_max)
        # Se define vértice inferior derecho de la img
        v_inf_der = (new_x_pos, new_y_pos)
        # Se calcula el vértice superior izquierdo de la img
        v_sup_izq = (new_x_pos - new_x_size, new_y_pos - new_y_size)

        # Se define padding adicional a la imagen
        padding = 4 # píxeles

        # Se detecta intersección entre otras imágenes
        c2 = (v_sup_izq[0] - padding, v_sup_izq[1] - padding, v_inf_der[0] + padding , v_inf_der[1] + padding)
        d2 = Rectangle(c2[0],c2[1],c2[2],c2[3])
        for old_coord in img_reg:
          c1 = np.asarray(old_coord)
          d1 = Rectangle(c1[0],c1[1],c1[2],c1[3])
          #Se evalúa si la intersección entre ambas imágenes es distinto de vacío
          if area(d1,d2) is None:
            do_intersect = False # No se detectó intersección
          else:
            do_intersect = True # Se detectó intersección
            n_colisiones = n_colisiones + 1
            break # se detiene ciclo for y se vuelve a generar coordenadas

      # Una vez se verifició que no hay colisión, se guarda coordenadas en registro
      img_reg.append(c2)

      # Se agrega máscara circular si do_Mask es True
      if do_Mask:
        #Se aplica máscara circular
        lum_img = Image.new('L',[new_x_size,new_y_size] ,0) 
        draw = ImageDraw.Draw(lum_img)
        draw.pieslice([(0,0),(new_x_size,new_y_size)],0,360,fill=255)
        # Se pega la imagen en el fondo
        background.paste(face, (new_x_pos, new_y_pos),lum_img)

      # Se abre archivo de texto del respectivo rostro
      label_name = f"{obj_img_list[i+j]}"
      #print(f"label_name = {label_name[:-4]}.txt")
      face_label = open(f'/content/AffectNet_dataset/obj/{label_name[:-4]}.txt','r')
      parameters = face_label.readlines() # Se lee la línea con los parámetros del rostro
      parameters = parameters[0].split(" ") # Se separan los parámetros y se guardan en forma de lista
      parameters[1] = ((float(parameters[1])*new_x_size)+new_x_pos)/res_mosaico[0]
      parameters[2] = ((float(parameters[2])*new_y_size)+new_y_pos)/res_mosaico[1]
      parameters[3] = ((float(parameters[3])*new_x_size))/res_mosaico[0]
      parameters[4]  = ((float(parameters[4])*new_y_size))/res_mosaico[1]
      # Se genera string con parámetros y se escribe en etiqueta de imagen generada
      parameters_line = ' '.join(map(str,parameters))
      # Se escriben los parámetros en el archivo de texto
      background_label.write(f"{parameters_line}\n")

      face_label.close() # Se cierra el archivo

  # Se guarda imagen de fondo modificado
  background.save(f"{j}.jpg")

  # Se guarda archivo de texto
  background_label.close()

  # Se incrementa contador de imágenes con las imágenes procesadas
  j = j + n_img_por_mosaico

  #display(background)
  #break
print("[INFO]: Conjunto de entrenamiento generado correctamente.")

In [ ]:
# Se conprime la carpeta con las imágenes generadas
%cd ../
!zip -r obj.zip obj

In [ ]:
# Se copia la base generada a la unidad de GDrive
!cp /content/Mosaic_Dataset/obj.zip /mydrive/Mosaic_Proc_AffectNet_YOLOv4_v2/obj.zip

### 2.4. Procesamiento conjunto de prueba
Ahora se procesa la base de datos para prueba a partir del conjunto pre definido de la base de datos descargada (archivo test.zip).

In [ ]:
# Se cambia directorio para generar conjunto de prueba
%cd /content/Mosaic_Dataset/test/

In [ ]:
# Se genera lista con nombres de archivos en conjunto de prueba
obj_img_list = list() # se inicializa lista para imágenes
# Se agregan los nombres de las imágenes a la lista
for file in os.listdir("/content/AffectNet_dataset/test/"):
    if file.endswith(".jpg"):
        obj_img_list.append(file)
# Se imprime número de imágenes encontradas en conjunto de prueba
print(f'[INFO]: Número de imágenes encontradas en set de prueba: {len(obj_img_list)}')
# Se reordena la lista con nombres de manera aleatoria
random.shuffle(obj_img_list) # se reordena la lista

In [ ]:
# Se crea contador para llevar cuenta del número de imágenes restantes en el conjunto de entrenamiento
img_restantes = len(obj_img_list)
# Se establece número de imágenes por mosaico
n_img_por_mosaico = 7 # este valor debe ser ingresado por el usuario
# Se establece resolución de imagen de salida
res_mosaico = [1024,768] # [width, height]

# Aplicar máscara?
do_Mask = True

In [ ]:
from IPython.core.display import Javascript
# Se definen tamaños máximos y mínimos para las imágenes de rostros
max_face_res = [90, 90] # [ancho, alto]
min_face_res = [25, 25] # [ancho, alto]
# Proporción de imagenes por grupo:
size_L = 0.1                    # grupo grande
size_M = 0.32                   # grupo mediano
size_S = (1 - size_L - size_M)  # grupo pequeño
# Se calcula el número de imágenes para la proporción definida
n_L = 1
n_M = 1
n_S = 1
# Proporción del fondo por sección (S, M, L)
b_size_L = 0.3
b_size_M = 0.3
b_size_S = (1 - b_size_L - b_size_M)
# Se calcula el número de píxels verticales del fondo que corresponde a cada proporción
b_L = int(b_size_L*res_mosaico[1])
b_M = int(b_size_M*res_mosaico[1])
b_S = res_mosaico[1] - b_L - b_M

# Se inicializa lista con registro de coordenadas
img_reg = list()
neg_img_counter = 0

# Se inicializa variable contadora de imágenes generadas
j = 0
# Se comienza con la generación de imágenes 
while j < len(obj_img_list):

  # se crea archivo de texto para guardar anotaciones (etiqueta)
  background_label = open(f'{j}.txt', 'w')

  # Se crea fondo negro
  background = Image.new('RGB', res_mosaico, (0, 0, 0))

  #--------------------EJEMPLOS NEGATIVOS------------------------------------------------------------------
  # Se define número de ejemplos negativos a agregar por mosaico generado
  n_ejem_neg_por_mosaic = 10

  # Se inicializa lista con registro de vértices de imágenes ya colocadas
  img_reg = [(0, 0, 2, 2)]

  # Se agregan los ejemplos negativos de manera aleatória
  for ji in range(n_ejem_neg_por_mosaic):
    # si el contador igualó o sobrepasó el máximo, se reinicia
    if ji+neg_img_counter >= len(neg_img_list): neg_img_counter = 0
    # Se abre imagen de ejemplo negativo a agregar
    ejem_neg = Image.open(f'/content/Ejemplos_negativos/{neg_img_list[ji + neg_img_counter]}')
    
    do_intersect = True # variable de verificación
    # Se genera posición nueva y se verifica la no intersección
    while do_intersect:
        # Se genera propuesta de coordenadas aleatoriamente
        neg_pos_x = random.randint(0, res_mosaico[0])
        neg_pos_y = random.randint(0, b_S + b_M)
        # Se define vértice inferior derecho de la img
        v_inf_der = (neg_pos_x, neg_pos_y)
        # Se calcula el vértice superior izquierdo de la img
        v_sup_izq = (neg_pos_x - ejem_neg.size[0], neg_pos_y - ejem_neg.size[1])
        # Se define padding adicional a la imagen
        padding = 4 # píxeles
        # Se detecta intersección entre otras imágenes
        c2 = (v_sup_izq[0] - padding, v_sup_izq[1] - padding, v_inf_der[0] + padding , v_inf_der[1] + padding)
        d2 = Rectangle(c2[0],c2[1],c2[2],c2[3])
        for old_coord in img_reg:
          c1 = np.asarray(old_coord)
          d1 = Rectangle(c1[0],c1[1],c1[2],c1[3])
          #Se evalúa si la intersección entre ambas imágenes es distinto de vacío
          if area(d1,d2) is None:
            do_intersect = False # No se detectó intersección
          else:
            do_intersect = True # Se detectó intersección
            n_colisiones = n_colisiones + 1
            break # se detiene ciclo for y se vuelve a generar coordenadas
        # Una vez se verifició que no hay colisión, se guarda coordenadas en registro
        #img_reg.append(c2)

    # Se agrega el ejemplo negativo al fondo
    background.paste(ejem_neg, (neg_pos_x, neg_pos_y))
    # Se cierra imagen abierta con ejemplo negativo
    ejem_neg.close()
  
  # se agrega el número de ejemplos negatios agregados al contador
  neg_img_counter = neg_img_counter + n_ejem_neg_por_mosaic
  #-----------------------EJEMPLOS NEGATIVOS--------------------------------------------------------------

  # Contador de colisiones
  n_colisiones = 0
  # Se verifica si se llegó a las últimas imágenes
  if (len(obj_img_list) - j) < n_img_por_mosaico:
    n_img_por_mosaico = len(obj_img_list) - j # se reduce número de rostros por imagen generada
  
  # Se agregan las fotos con rostros al fondo
  for i in range(n_img_por_mosaico):
    # Se abre imagen con rostro
    face = Image.open(f'/content/AffectNet_dataset/test/{obj_img_list[i+j]}')

    for ii in range(3):
      if ii < n_L:
        # número de imágenes por sección
        n_img = n_L
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = max_face_res[0]
        yface_min = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = max_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L)
        y_max = (res_mosaico[1] - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      elif ii >= n_L and ii < (n_M + n_L): 
        # número de imágenes por sección
        n_img = n_M
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L - b_M)
        y_max = (res_mosaico[1] - b_L - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      else:
        # número de imágenes por sección
        n_img = n_S
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = min_face_res[0]
        xface_max = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = min_face_res[1]
        yface_max = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = 0
        y_max = (res_mosaico[1] - b_L - b_M - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      # Se calcula nueva dimensión de imagen con rostro
      new_x_size = random.randint(xface_min, xface_max)
      #new_y_size = random.randint(yface_min, yface_max) # se pierde aspect ratio :(
      #Se calcula el alto para conservar aspect ratio
      wpercent = (new_x_size/float(face.size[0]))
      new_y_size = int((float(face.size[1])*float(wpercent)))
      # Se escala la imagen seleccionada
      face = face.resize((new_x_size, new_y_size), resample=PIL.Image.BILINEAR)
      # Se calcula posición de la imagen asegurando la no intersección con imágenes anteriores
      do_intersect = True # variable de verificación
      # Se genera posición nueva y se verifica la no intersección
      while do_intersect:
        # generación de coordenadas nuevas
        new_x_pos = random.randint(x_min, x_max)
        new_y_pos = random.randint(y_min, y_max)
        # Se define vértice inferior derecho de la img
        v_inf_der = (new_x_pos, new_y_pos)
        # Se calcula el vértice superior izquierdo de la img
        v_sup_izq = (new_x_pos - new_x_size, new_y_pos - new_y_size)

        # Se define padding adicional a la imagen
        padding = 4 # píxeles

        # Se detecta intersección entre otras imágenes
        c2 = (v_sup_izq[0] - padding, v_sup_izq[1] - padding, v_inf_der[0] + padding , v_inf_der[1] + padding)
        d2 = Rectangle(c2[0],c2[1],c2[2],c2[3])
        for old_coord in img_reg:
          c1 = np.asarray(old_coord)
          d1 = Rectangle(c1[0],c1[1],c1[2],c1[3])
          #Se evalúa si la intersección entre ambas imágenes es distinto de vacío
          if area(d1,d2) is None:
            do_intersect = False # No se detectó intersección
          else:
            do_intersect = True # Se detectó intersección
            n_colisiones = n_colisiones + 1
            break # se detiene ciclo for y se vuelve a generar coordenadas

      # Una vez se verifició que no hay colisión, se guarda coordenadas en registro
      img_reg.append(c2)

      # Se agrega máscara circular si do_Mask es True
      if do_Mask:
        #Se aplica máscara circular
        lum_img = Image.new('L',[new_x_size,new_y_size] ,0) 
        draw = ImageDraw.Draw(lum_img)
        draw.pieslice([(0,0),(new_x_size,new_y_size)],0,360,fill=255)
        # Se pega la imagen en el fondo
        background.paste(face, (new_x_pos, new_y_pos),lum_img)

      # Se abre archivo de texto del respectivo rostro
      label_name = f"{obj_img_list[i+j]}"
      #print(f"label_name = {label_name[:-4]}.txt")
      face_label = open(f'/content/AffectNet_dataset/test/{label_name[:-4]}.txt','r')
      parameters = face_label.readlines() # Se lee la línea con los parámetros del rostro
      parameters = parameters[0].split(" ") # Se separan los parámetros y se guardan en forma de lista
      parameters[1] = ((float(parameters[1])*new_x_size)+new_x_pos)/res_mosaico[0]
      parameters[2] = ((float(parameters[2])*new_y_size)+new_y_pos)/res_mosaico[1]
      parameters[3] = ((float(parameters[3])*new_x_size))/res_mosaico[0]
      parameters[4]  = ((float(parameters[4])*new_y_size))/res_mosaico[1]
      # Se genera string con parámetros y se escribe en etiqueta de imagen generada
      parameters_line = ' '.join(map(str,parameters))
      # Se escriben los parámetros en el archivo de texto
      background_label.write(f"{parameters_line}\n")

      face_label.close() # Se cierra el archivo

  # Se guarda imagen de fondo modificado
  background.save(f"{j}.jpg")

  # Se guarda archivo de texto
  background_label.close()

  # Se incrementa contador de imágenes con las imágenes procesadas
  j = j + n_img_por_mosaico

  #display(background)
  #break
print("[INFO]: Conjunto de prueba generado correctamente.")

In [ ]:
# Se conprime la carpeta con las imágenes generadas
%cd ../
!zip -r test.zip test

In [ ]:
# Se copia la base comprimida a la unidad de GDrive
!cp /content/Mosaic_Dataset/test.zip /mydrive/Mosaic_Proc_AffectNet_YOLOv4_v2/test.zip

In [ ]:
print("[INFO]: Programa finalizado.")

In [ ]:
#Se cuentan las imágenes creadas en los datasets generados
img_list_test = [x for x in glob.glob("/content/Mosaic_Dataset/test/*.jpg")]
print(f"[INFO]: Imágenes (mosaicos) de validación generados: {len(img_list_test)}")
img_list_train = [x for x in glob.glob("/content/Mosaic_Dataset/obj/*.jpg")]
print(f"[INFO]: Imágenes (mosaicos) de entrenamiento generados: {len(img_list_train)}")